In [9]:
!pip install huggingface-hub -Uqq

In [5]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./model")
local_model_path.mkdir(exist_ok=True)
model_name = "bigscience/bloomz-7b1-mt"
commit_hash = "13e9b1a39fe86c8024fe15667d063aa8a3e32460"

In [ ]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

s3://sagemaker-us-east-1-106839800180/LLM-RAG/workshop/model/

In [2]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [3]:
bucket

'sagemaker-us-east-1-106839800180'

In [16]:
s3_model_prefix = "LLM-RAG/workshop/model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/deploy_code"
s3_code_prefix

'LLM-RAG/workshop/deploy_code'

In [ ]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

In [8]:
inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed", region=region, version="0.21.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")


# inference_image_uri = (
#     f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117"
# )

# print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117


In [10]:
!mkdir -p deploy_code

In [56]:
%%writefile deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location)
   
    model = AutoModelForCausalLM.from_pretrained(
        model_location, 
        device_map="balanced_low_0", 
        #load_in_8bit=True
    )
    model.requires_grad_(False)
    model.eval()
    
    return model, tokenizer


model = None
tokenizer = None
generator = None


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    
    # preprocess
    input_ids = tokenizer(input_sentences, return_tensors="pt").input_ids
    # pass inputs with all kwargs in data
    if params is not None:
        outputs = model.generate(input_ids, **params)
    else:
        outputs = model.generate(input_ids)

    # postprocess the prediction
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    result = {"outputs": prediction}
    return Output().add_as_json(result)

Overwriting deploy_code/model.py


In [57]:
%%writefile deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=4
option.s3url = s3://sagemaker-us-east-1-106839800180/LLM-RAG/workshop/model/

Overwriting deploy_code/serving.properties


In [58]:
!rm model.tar.gz
!tar czvf model.tar.gz deploy_code

deploy_code/
deploy_code/model.py
deploy_code/.ipynb_checkpoints/
deploy_code/.ipynb_checkpoints/serving-checkpoint.properties
deploy_code/.ipynb_checkpoints/model-checkpoint.py
deploy_code/serving.properties


In [59]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-1-106839800180/LLM-RAG/workshop/deploy_code/model.tar.gz


In [60]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"bloomz-7b1-mt") # Append a timestamp to the provided string
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

bloomz-7b1-mt-2023-04-13-11-02-25-553
Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117
Created Model: arn:aws:sagemaker:us-east-1:106839800180:model/bloomz-7b1-mt-2023-04-13-11-02-25-553


In [61]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.12xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:106839800180:endpoint-config/bloomz-7b1-mt-2023-04-13-11-02-25-553-config',
 'ResponseMetadata': {'RequestId': '49275446-d17d-49f5-b8b3-50e34a566719',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '49275446-d17d-49f5-b8b3-50e34a566719',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '125',
   'date': 'Thu, 13 Apr 2023 11:02:39 GMT'},
  'RetryAttempts': 0}}

In [62]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-1:106839800180:endpoint/bloomz-7b1-mt-2023-04-13-11-02-25-553-endpoint


In [63]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:106839800180:endpoint/bloomz-7b1-mt-2023-04-13-11-02-25-553-endpoint
Status: InService


In [65]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  #"early_stopping": True,
  #"length_penalty": 2.0,
  "max_new_tokens": 50,
  "temperature": 0,
  "min_length": 10,
  "no_repeat_ngram_size": 2,
}

CPU times: user 585 µs, sys: 3.35 ms, total: 3.93 ms
Wall time: 11 ms


In [66]:
prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
请问AWS Clean Rooms是多方都会收费吗？
"""

response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

'{\n  "outputs":"AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。\\n请问AWS Clean Rooms是多方都会收费吗？\\n 不"\n}'

In [67]:
prompts1 = """请问AWS Clean Rooms是多方都会收费吗？"""

response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

'{\n  "outputs":"请问AWS Clean Rooms是多方都会收费吗？ 是的"\n}'

In [68]:
prompts1 = """请问AWS Clean Rooms的一个协作中可以有多少个参与方?"""

response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

'{\n  "outputs":"请问AWS Clean Rooms的一个协作中可以有多少个参与方? 最多 100"\n}'

In [71]:
prompts1 = """AWS Clean Rooms 的FAQ文档有提到, Question: AWS Clean Rooms的一个协作中可以有多少个参与方? Answer: 每个协作AWS Clean Rooms最多支持五名参与方。
请问AWS Clean Rooms的一个协作中可以有多少个参与方?"""

response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

'{\n  "outputs":"AWS Clean Rooms 的FAQ文档有提到, Question: AWS Clean Rooms的一个协作中可以有多少个参与方? Answer: 每个协作AWS Clean Rooms最多支持五名参与方。\\n请问AWS Clean Rooms的一个协作中可以有多少个参与方? 五"\n}'

In [ ]:
input="""
AWS 文档中提到: Q：在中国区是否可用？
A： 目前没有落地中国区的时间表，已经在以下区域推出：美国东部（弗吉尼亚州北部）、美国东部（俄亥俄州）、美国西部（俄勒冈州）、亚太地区（首尔）、亚太地区（新加坡）、亚太地区（悉尼）、亚太地区（东京）、欧洲地区（法兰克福）、欧洲地区（爱尔兰）、欧洲地区（伦敦）和欧洲地区（斯德哥尔摩）

Q: 为什么在合成的小数据集上第一次查询的时候需要几分钟才返回？
A：第一次查询的时候是因为调度和拉起资源的影响，一般第二次查询就会变快。 但过一段时间后，资源释放后这个问题又会出现。在资源拉起期间是不进行收费的。

Q：能支持多大规模数据的查询？查询速度怎么样？
A：能支持TB/GB级数据的查询。 一般查询延迟为几十秒到几分钟。默认计算容量为32 CRPUs, 目前这个默认计算容量不可设置，但是roadmap中未来打算让用户可以进行设置。(Slack中Ryan 提到，如果引擎中任务有积压，它能够scale up）

Q: AWS Clean Rooms 是如何计费的？
A: 按照CRPU-hour单价进行计费，每个查询默认计算容量为32 CRPUs。 金额 = (0.125 hours x 32 CRPUs * $0.656 per CRPU-hour) , 有1分钟的最小计费时间。头12个月内，会有9CRPU hours的免费额度。

Q: AWS Clean Rooms 从哪里可以看到CRPU-hours的用量？
A: AWS Clean Rooms 本身的workspace中无法查看，可以在AWS Billing/Bills 中查看Usage Quantity得到该信息。

Q: 目前可以支持什么数据源的接入？ 
A: 目前只支持S3，其他数据源近期没有具体计划。

Q: 一个协作中，最大的并发查询数是多少？
A: 5个

Q: 我们如何说服客户相信洁净室的安全性和正确性？我在这里的大多数合规计划中都看不到Clean Rooms， https://aws.amazon.com/compliance/services-in-scope/ ？
A: 正在加入这些合规计划的进程中。


使用方法与限制相关:

Q: 数据源必须在AWS上么？
A: 对，目前必须在AWS上，而且必须是同一个region。

Q: 数据是如何进入到S3？
A: 需要数据的持有者，把数据上传到S3上，然后再用Glue爬去下，拿到表的schema。这样才能关联到AWS CleanRooms。

Q: 这些安全控制权限只是作用与分析么？能够改动它方的数据么？
A: 对，只作用于分析，不能修改对方数据

Q: 协作方的数据会移动么？协作方的原始数据会集中到Clean rooms 吗？
A: 在联合分析获取他方数据时数据存在移动，但协作方的原始数据并不会存住在clean room内，clean room并不是一个物理存贮空间。

Q: 是否发起者和数据贡献者都会被收费？
A: 是单方收费，只有查询的接收方会进行收费。

Q: 数据贡献方的S3， 会产生API会产生调用次数收费么？
A: 会， Glue Data Catalog API 的调用也会被收费， 如果加密数据用了KMS-CMK也会被相应的收费。

Q: 是否有一个强约束，两方的数据中一定要一个join字段才能够进行分析？
A: List 和Aggregation两种不同的分析规则下有区别， List 只能支持重合用户的，所以必须要有关联字段。Aggregation可以支持你仅仅去查询对方的数据，这种情况下，是可以不指定关联字段的。

Q: 如果已经在用Athena，S3桶中已经有数据了，是否能基于S3中这个数据就地加入AWS Clean Room?
A: 是的，就地就能加入clean room, 这个S3桶就是一般的S3桶，并没有任何特殊。但这个S3路径不能注册到AWS Lake Formation中。

Q: 是否能通过SDK也就是代码来调用Clean room的联合分析
A: 可以，可以参考代码 https://gitlab.aws.dev/rmalecky/aws-clean-rooms-notebooks/-/blob/main/single_collaborator_aggregation.ipynb

Q: 在输出分析结果的时候，能否按照字段进行分区？
A: 目前不能

Q: 最大的参与方是多少？ 如果超过了限制怎么办？
A: 目前5个参与方为最大限制，这个是软性的限制，slack频道中有披露最大支持的硬限制为10.


数据加密相关：

Q: AWS clean rooms 用的什么加密算法？
A: C3R，是aws开源加密代码库。提供了C3R Client(一个可执行的Jar包)，目前仅支持对csv和parquet文件格式进行加密，后续可能会支持更多格式。 由于clean room把所有的字段分成三种类型： 指纹列(fingerprint column), 密封列(sealed column), 明文列(cleartext column), 他们的加密方式有所不同，C3R client 会使用AES-GCM加密算法对sealed字段进行加密，会使用HMAC(Hash-based Message Authentication Code)来对fingerprint字段进行加密。

Q: 加密过程中有密文落地么？如果有，密文存在哪里？
A：由于C3R是客户端加密，所以clean room 关联S3中的数据已经是加密后的密文。

Q: 数据上传到S3的过程中，有两种加密方式Server Side 和 Client Side加密，如果客户的安全等级比较高，在数据上传之前做了加密，再想交给clean room 去处理，之前提到的C3R的加密方式，具体是一个怎么样的流程呢？
A: 首先Clean room 对于Server Side的加密是透明的，无需额外处理。Clean rooms 不支持S3的客户端加密，必须采用C3R客户端进行加密，加密完成以后把数据上传到S3桶，后续流程和不加密的流程是一致的。 加密这步需要比较多的手工操作，包括：

* 加密前需要创建好collaboration(协作), 得到collaboration_id后续在加密中需要提供
* 利用openssl 生成32位密钥并分享给其他协作方。
* 加密过程中需要指定哪些字段为指纹列(fingerprint column), 密封列(sealed column), 明文列(cleartext column)


Q: AWS Clean Room在安全计算方面，应该归属哪一类？
A: (待补充)

Q: 是否所有的字段都可以进行加密？
A: 对于sealed和 fingerprint字段，只有string字段类型被支持。对于csv文件，C3R的客户端处理任何值都作为UTF-8编码的文本，加密前不会做任何其他的前置处理。对于parquet文件，对sealed和 fingerprint字段，如果出现非string的字段，会直接报错。C3R 客户端不能处理parquet中的复杂字段比如struct。

Q: 如果要对某个字段进行求和，求平均的数据计算，是否可以加密？
A: 不能，只能作为cleartext明文列

Q: 如果想要通过某些字段进行where过滤，这些字段应该是什么类型？
A: 只能是cleartext明文列

Q: C3R客户端是否有实现任何non-standard的加密算法?
A: 基本都是标准化的算法，除了一个HKDF(一种密钥推导函数)的实现(来自RFC5869), 但是使用的是java标准加密库中的MAC算法。


宏观问题：

Q: AWS Clean rooms 与 AMC的区别与联系是什么？
A: AMC是一个专门服务与Amazon Ads的clean room应用，它是并且将持续是唯一的服务于Amazon Ads客户的应用服务。AWS Clean Rooms 是一个云分析服务，会服务于各个行业的数据合作需求。2023年, AMC 将会把自己的查询引擎和计算基础设置迁移到AWS Clean Rooms服务，将会帮助AMC更方便的服务于客户(他们将不再需要把自己第一方数据上传到AMC，在AWS S3上即可使用).

Q: AWS Clean rooms 与 其他的clean room服务商的区别？
A: AWS Clean rooms 覆盖的客户范围更广。而其他的服务商客户范围相对小，需要把数据移动到他们的平台上。AWS Clean rooms则无需数据移动。(Bastion 2021年announce的时候，其他的云厂商比如(google cloud 和 microsoft azure) 还没有通用场景的clean room solution，snowflake 有，功能上有区别，只允许data provider 提供预先固定的SQL，Bastion灵活性更好。snowflake要求数据必须进入他们的数仓，aws在S3 即可)。

Q: 有哪些典型的应用场景？
A: 包含多个行业，下面提供部分参考

* 广告营销领域：
    * 需要进行广告营销活动，流量平台方需要给广告主或者营销方他们的广告点击数据和展现数据进行分析，但是不能提供用户级别的信息。
    * 典型客户：营销方或广告主 P&G, Barclays，媒体-流量平台 Amazon Ads, Comcast, NBC Universal
* 零售领域
    * 银行和零售商需要获取他们的重叠用户，用于进行联合的市场活动，但不要把客户的其他信息暴露给彼此。（比如非场合用户）
* 医疗健康领域 (结合之前的HCLS Slides， 并不是来自Bastion)
    * 药厂和医院之间，药厂需要医院的病历数据； 药厂和外包的研发机构之间需要进行数据的共享。
    * 典型客户：Change Healthcare, AstraZeneca 
* 其他领域的一些典型的客户
    * 数据服务商 Foursquare ，Nielsen， IRI， 
    * 其他 Cars.com
    
内部信息(来自Bastion PRFAQ(2021年))[Internal Only]:

Q: AWS Clean Rooms 的 data catalog 是如何实现的？ data sharing permission 是如何实现的？
A: 都是利用了AWS Lake formation, AWS Clean Rooms 里 SQL中字段级别的限制约束，是通过一种new class of AWS Lake formation permission 来实现的。

Q: 可以在哪些地方进行Clean room的联合分析？
A: 可以在clean room 的 workspace， 也可以在Redshift workspace （Note: 从目前发布产品文档上并没有，但是说明背后的引擎就是redshift severless)

Q: 数据提供方如果对联合分析的收益方进行收费，或者实现一个数据授权的合同？
A: 需要通过AWS Data Exchange 来进行 （Note: 目前AWS Clean Rooms并没有体现）

Q: AWS Clean Rooms 与 AWS Data Exchange 是什么关系？
A: AWS Clean Rooms 可以通过AWS Data Exchange 去浏览和寻找可用数据的合作方。 他是AWS Data Exchange的更近一步的服务，提供了可控(多种约束限制)和可审计的数据合作方式

Q: AWS Clean Rooms中是否支持视图？
A: 允许客户在clean room 创建视图，并且在AWS Clean Rooms中保存物化视图，一旦退出协作，AWS Lake formation permission 将会被撤销，这些物化视图会被删除。（Note: 目前AWS Clean Rooms并没有体现）

Q: 如果数据合作方没有aws account，能否支持？
A: 目前这个版本不支持，后续的版本可能会考虑（NBC Universal 希望对于没有aws账号另外一方的数据可用）

Q: 是否能够支持这个协作中，仅仅允许指定运行固定的SQL？
A: 可以，可以利用query template来做（Note: 目前AWS Clean Rooms并没有体现）

Q: AWS Clean Rooms可以让数据贡献者提供一些样例数据进行预览么？
A: 可以这么做，可以提供一些没有任何约束的示例数据给用户（Note: 目前AWS Clean Rooms并没有体现）

Q: 当一个数据贡献者的数据发生更新后会怎么样？
A: 它是一种live的共享，任何更新会立刻反映到联合分析的结果中

Q: AWS Clean Rooms 未来有哪些前进的方向？
A: 主要有四个方向：

1. Identity matching 身份ID对齐 (Note: 目前这项在官方的PPT在有体现)
2. 对隐私攻击的防护， 有些查询即使是一些聚合分析，仍然可能探查到个人的信息
    1. 限制访问同一块范围数据的query的数量
    2. 采用差分隐私(Differential Privacy) 结果中添加噪声(会影响分析的精度)，Morgen Stanley 作为这个功能的beta用户
3. 机器学习，P&G 表现出这方面的需求， 应该也是基于表格数据的模型，可能是流失预测，人群聚类等场景
4. 和DSP的集成，直接把激活用户ID给到对接的DSP，而不通过cleanroom的数据接收方


Q: Service Team 有哪些相关的同事？
A:  Horne, Bill <bgh@amazon.com>, Rababy, Bethany <rababyb@amazon.com>, Malecky, Ryan <rmalecky@amazon.com>, Malik, Mohsen <mmohsen@amazon.com>, Tanna, Shamir <tannas@amazon.com>

请问AWS Clean rooms 在中国区是否可用？
"""

response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": input,
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

In [79]:
input="""
AWS 文档中提到: 
可用性和性能相关：

Question: 在中国区是否可用？
Answer: 目前没有落地中国区的时间表，已经在以下区域推出：美国东部（弗吉尼亚州北部）、美国东部（俄亥俄州）、美国西部（俄勒冈州）、亚太地区（首尔）、亚太地区（新加坡）、亚太地区（悉尼）、亚太地区（东京）、欧洲地区（法兰克福）、欧洲地区（爱尔兰）、欧洲地区（伦敦）和欧洲地区（斯德哥尔摩）

Question: 为什么在合成的小数据集上第一次查询的时候需要几分钟才返回？
Answer: 第一次查询的时候是因为调度和拉起资源的影响，一般第二次查询就会变快。
...

请问AWS Clean rooms 为什么第一次运行很慢？
"""

response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": input,
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

'{\n  "outputs":"\\nAWS 文档中提到: \\n可用性和性能相关：\\n\\nQuestion: 在中国区是否可用？\\nAnswer: 目前没有落地中国区的时间表，已经在以下区域推出：美国东部（弗吉尼亚州北部）、美国东部（俄亥俄州）、美国西部（俄勒冈州）、亚太地区（首尔）、亚太地区（新加坡）、亚太地区（悉尼）、亚太地区（东京）、欧洲地区（法兰克福）、欧洲地区（爱尔兰）、欧洲地区（伦敦）和欧洲地区（斯德哥尔摩）\\n\\nQuestion: 为什么在合成的小数据集上第一次查询的时候需要几分钟才返回？\\nAnswer: 第一次查询的时候是因为调度和拉起资源的影响，一般第二次查询就会变快。\\n...\\n\\n请问AWS Clean rooms 为什么第一次运行很慢？\\n第一次运行的原因是调度的拉起的资源影响"\n}'

In [75]:
len(input)

4307

In [ ]:
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

In [54]:
!aws sagemaker delete-endpoint --endpoint-name bloomz-7b1-mt-2023-04-13-09-54-18-256-endpoint

In [55]:
!aws sagemaker delete-endpoint-config --endpoint-config-name bloomz-7b1-mt-2023-04-13-09-54-18-256-config